In [ ]:
# %pip install -q rasterio
# %pip install -q rio-cogeo
# %pip install -q owslib
# %pip install -q rioxarray
# %pip install -q geopandas
# %pip install -q folium

In [1]:
from maap.maap import MAAP
maap = MAAP()
import json
from IPython.display import display, Image
import ipycmc
w = ipycmc.MapCMC()
from pprint import pprint
import rasterio as rio
from rasterio.plot import show
import requests
import os
from owslib.wcs import WebCoverageService
import rioxarray
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
import folium

# MAAP Version 1 Delivery 2 (V1D2) Data System Demo Notebook

## Responding to User Working Group Requests from working sessions and the usability study

### What were the recommendations from the MAAP Usability Study?

* Create more documentation and examples
* Enable visualization data on a map
* Ways to handler large amounts of data

### How did the data system respond? (Demo Outline)

For V1D2, the MAAP data system made updates to address the recommendations of the usability study. This notebook demonstrates those updates. In it we:

* Demonstrate using the **[documenation site](https://maap-project.readthedocs.io/en/latest/)** to:
  - query by additional attributes
  - visualize data on a map
* Demonstrate users can handle **large amounts of data** using cloud-optimized formats and OGC standards with:
    * NASA Shuttle Radar Topography Mission Global 1 arc second V003 (COGs)
      - **Visualization** - WMTS with the Dynamic Tiler API + MAAP API
    * AfriSAR UAVSAR Coregistered SLCs Generated Using NISAR Tools (COGs)
      - **Query** - WCS via EDAV
    * ATLAS/ICESat-2 L3A Land and Vegetation Height V003 (EPT)
      - **Visualization** - 3D Tiles 
      - **Query** - Features
 

# Using Documentation

We can use [Searching by Additional Attributes](https://maap-project.readthedocs.io/en/latest/search/granules.html?highlight=additional#Searching-by-Additional-Attributes) documentation to search by the requested additional attributes `direction` and `polarization`.

In [ ]:
results = maap.searchGranule(orbit_dir = "ASCENDING", limit=100)
pprint(f'Got {len(results)} results')
print(json.dumps(results[0], indent=2))

In [ ]:
results = maap.searchGranule(polarization = "HH", limit=100)
pprint(f'Got {len(results)} results')
print(json.dumps(results[0], indent=2))

We can use [Visualizing Web Map Tile Service (WMTS) Layers
](https://maap-project.readthedocs.io/en/latest/visualization/using_pycmc.html) documentation to visualize data on a map.

In [ ]:
# Import the ipycmc module
import ipycmc
# utilize the CMC widget
w = ipycmc.MapCMC()
w

In [ ]:
w.load_layer_config("https://api.maap-project.org/api/wmts/GetCapabilities", "wmts/xml")

# Handling Large Amounts of Data

## SRTM Cloud-Optimized GeoTiffs (COGs)

SRTM (Shuttle Radar Topography Mission) obtained elevation data on a near-global scale using radar interferometry. The SRTMGL1 contains elevation data in 1° X 1° tiles at 1 arc second (about 30 meters) resolution.

During this delivery, SRTMGL1 data product was made available in **Cloud-Optimized GeoTiff (COG)** format so that it can be dynammically visualizaed on a MAAP alongside the previoously published COGs of UAVSAR and LVIS.

## SRTM Visualization

In [ ]:
results = maap.searchGranule(short_name="SRTMGL1_COD", bounding_box="7.27,-5.24,17.00,3.48")
for r in results:
    granule_ur = r['Granule']['GranuleUR']
    w.load_layer_config(f"https://api.maap-project.org/api/wmts/GetCapabilities?granule_ur={granule_ur}", "wmts/xml")

## Querying UAVSAR AfriSAR using WCS

Let's send a request to the EDAV WCS and extract a subset of the data in GeoTiff format and save it to our workspace.

In [ ]:
c = maap.searchCollection(short_name="AfriSAR_UAVSAR_Coreg_SLC")[0]
c['Collection']['Description']

In [ ]:
# Configure the WCS source
EDAV_WCS_Base = "https://edav-wcs.adamplatform.eu/wcs"
wcs = WebCoverageService(f'{EDAV_WCS_Base}?service=WCS', version='2.0.0')

# Request the data from WCS
response = wcs.getCoverage(
    identifier=['uavsar_AfriSAR_v1_SLC'],
    format='image/tiff',
    filter='false',
    scale=1,
    subsets=[('Long',11.6,11.7),('Lat',-0.2,-0.1)]
)

# Save the results to file as a tif
results = "EDAV_example.tif"
with open(results, 'wb') as file:
    file.write(response.read())

Do a quick check that the data is valid, and contains spatial metadata. For fun we can check if it's a Cloud Optimized Geotiff.

In [ ]:
!gdalinfo {results}
!rio cogeo validate {results}

### Read the data and do some quick visual exploration.

In [ ]:
edav_x = rioxarray.open_rasterio(results)
edav_x.plot(cmap="gist_earth", figsize=(10, 8)).set_clim(0, 0.4)

# Handling Large Amounts of (Point Cloud) Data (Continued)

## ATL08 v003 Entwine Point Tiles (EPT)

During V1D2, the data team made the ATL08 IceSAT-2 Dataset available in [Entwine Point Tile (EPT)](https://entwine.io/entwine-point-tile.html) format.

You can find additional documentation for the variables stored in the MAAP ATL08 data store on [the documentation site: Querying ATL08 Entwine Point Tiles](https://maap-project.readthedocs.io/en/latest/query/testing-ept-stores.html).

### What are Entwine Point Tiles?

Entwine Point Tiles are a cloud-optimized octree data format for storing and visualizing massive point clouds efficiently. This format is gaining a lot of momentum and interest with an active development community.


### Visualizing with potree

Potree is a tool for visualizing EPT stores directly.

https://potree.entwine.io/data/view.html?r=%22https://cumulus-map-internal.s3.amazonaws.com/file-staging/nasa-map/ATL08_ARD-beta___001/global/ept%22

In [ ]:
img_src = "images/Oct-28-2020 16-41-44.gif"
Image(url = img_src)

### Visualizing with the 3D Tiles Service (OGC)

Cesium is a 3D tool which can be used to visualize point clouds alongside 2D data using a 3D Tiles Service.

* [3D Tiles API Endpoint](https://api.maap.xyz/api/3d-tiles/ATL08_ARD-beta___001/global/ept/ept-tileset/tileset.json)
* [Demo using Cesium](http://cesium.entwine.io/?url=https://api.maap.xyz/api/3d-tiles/ATL08_ARD-beta___001/global/ept/ept-tileset/tileset.json)

In [ ]:
img_src = "images/Oct-28-2020 16-53-01.gif"
Image(url = img_src)

### Visualizing with ipyCMC

In [ ]:
w.load_layer_config("https://cmr.maap-project.org/search/concepts/G1200354094-NASA_MAAP.json", "json", {"handleAs": "vector-3d-tile"})

## Querying ATL08 EPT using the Features API

We can use the Features Service for some basic querying of the EPT Store.

> OGC API Features provides API building blocks to create, modify and query features on the Web.

[Documentation: OGC API - Features](https://www.ogc.org/standards/ogcapi-features)

### Query By Bounding Box

Using EPT and the features service delivers a response **100x faster** over the conventional way of subsetting this data (search, download, read and subset data). See the test in [subset-atl08-files.ipynb](subset-atl08-files.ipynb).

In [2]:
%%time
# Format a request to the API
api_url = "https://obnrh8ozt0.execute-api.us-east-2.amazonaws.com/collections/Global/items"

# Make a request for a bounding box over Peru
bbox="-77,-26,300,-73,0,500"
limit = 1000
payload = {
    "f": "json",
    "limit": limit,
    "bbox": bbox
}

r = requests.get(api_url, params = payload)
# Get the results directly into a Geo Data Frame (saving to file not required but recommended)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.shape
adf.head()

CPU times: user 147 ms, sys: 5.44 ms, total: 153 ms
Wall time: 5.55 s


,geometry,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,...,GpsTime,Red,Green,Blue,ScanChannel,ClassFlags,ElevationLow,HeightAboveGround,OffsetTime,OriginId
0,POINT Z (-75.58000 -8.80000 310.06000),-75.58,-8.80,310.06,0.0,1.0,1.0,0.0,0.0,0.0,...,1.240313e+09,10.0,95.0,101.0,0.0,0.0,289.127,42.708,41512918.0,29028.0
1,POINT Z (-74.17000 -11.57000 471.59000),-74.17,-11.57,471.59,0.0,1.0,1.0,0.0,0.0,0.0,...,1.245357e+09,27.0,107.0,111.0,0.0,0.0,488.210,25.316,46556779.0,37945.0
2,POINT Z (-73.48000 -10.88000 329.13000),-73.48,-10.88,329.13,0.0,1.0,1.0,0.0,0.0,0.0,...,1.258205e+09,25.0,106.0,110.0,0.0,0.0,314.425,35.354,59404688.0,58028.0
3,POINT Z (-74.16000 -10.18000 304.93000),-74.16,-10.18,304.93,0.0,1.0,1.0,0.0,0.0,0.0,...,1.258550e+09,12.0,96.0,101.0,0.0,0.0,302.224,20.358,59749777.0,58629.0
4,POINT Z (-73.51000 -9.49000 319.50000),-73.51,-9.49,319.50,0.0,1.0,1.0,0.0,0.0,0.0,...,1.266051e+09,17.0,100.0,105.0,0.0,0.0,296.021,27.648,67251450.0,71742.0


In [ ]:
m = folium.Map(
    location=[adf.centroid[0].y, adf.centroid[0].x],
    zoom_start=10,
    tiles='Stamen Terrain'
)


folium.GeoJson(
    adf,
    name = "geojson"
).add_to(m)

m

### Query by Granule Id

The UWG requested being able to query by granule id.

In [ ]:
granule_id = 'ATL08_20181014035224_02370107_003_01'
payload = {
    "f": "json",
    "origin": granule_id,
}

r = requests.get(api_url, params = payload)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.head()

## More query options with pdal

The [documentation](https://maap-project.readthedocs.io/en/latest/query/testing-ept-stores.html#PDAL-Pipelines) provides additional options for how to query with PDAL (Point Cloud Data Abstraction Library).

# Thank You!

**Big thanks to the ESA, JPL and MSFC dev teams.**

Aaron Kaulfus, Alex Mandel, Brian Satorius, Chuck Daniels, David Bitner, George Chang, Hai, Kaylin Bugbee, Maya DeBellis, Slesa Adhikari, Sam Ayers and Seth Vincent with whom none of this would be possible.

And thank you to the project scientists and user working group for providing such actionable feedback!